In [60]:
import csv
import os
from os import path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
# Define relative filepath in a subdirectory
file_path = "data/taylor_album_songs.csv"
data_directory = 'data'
os.makedirs(data_directory, exist_ok=True)

# Read file and print dataframe
data_taylor_album_songs = pd.read_csv(file_path)
print("Taylor Album Songs DataFrame:")
print(data_taylor_album_songs.head())

# Clean data and create a new csv
columns_to_drop = ['artist_name', 'artist_id', 'album_id', 'album_release_year', 'album_release_date_precision', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'disc_number', 'duration_ms', 'track_preview_url', 'track_uri', 'external_urls.spotify', 'key_name', 'mode_name', 'key_mode', 'track_href', 'is_local', 'analysis_url']
data_taylor_album_songs.drop(columns=columns_to_drop, axis=1, inplace=True)
values_to_remove = ["Taylor Swift", "Fearless (Taylor's Version)", "Speak Now (Taylor's Version)", "Red (Taylor's Version)", "1989 (Deluxe)", "reputation", "Lover", "folklore (deluxe version)", "evermore (deluxe version)", "Midnights (The Til Dawn Edition)"]
data_taylor_album_songs = data_taylor_album_songs[data_taylor_album_songs['album_name'].isin(values_to_remove)]
data_taylor_album_songs['lyrics'] = np.nan
cleaned_album_data = data_taylor_album_songs
data_taylor_album_songs.to_csv('data/cleaned_album_data.csv', index=False)

# Print column names and data types
print("Column names:", data_taylor_album_songs.columns)
print("Data types:")
print(data_taylor_album_songs.dtypes)

Taylor Album Songs DataFrame:
    artist_name               artist_id                album_id album_type  \
0  Taylor Swift  06HL4z0CvFAxyc27GXpf02  5eyZZoQEFQWRHkV2xgAeBw      album   
1  Taylor Swift  06HL4z0CvFAxyc27GXpf02  5eyZZoQEFQWRHkV2xgAeBw      album   
2  Taylor Swift  06HL4z0CvFAxyc27GXpf02  5eyZZoQEFQWRHkV2xgAeBw      album   
3  Taylor Swift  06HL4z0CvFAxyc27GXpf02  5eyZZoQEFQWRHkV2xgAeBw      album   
4  Taylor Swift  06HL4z0CvFAxyc27GXpf02  5eyZZoQEFQWRHkV2xgAeBw      album   

  album_release_date  album_release_year album_release_date_precision  \
0         2006-10-24                2006                          day   
1         2006-10-24                2006                          day   
2         2006-10-24                2006                          day   
3         2006-10-24                2006                          day   
4         2006-10-24                2006                          day   

   danceability  energy  key  ...  \
0         0.580   0.491  

In [62]:
# Define relative filepath in a subdirectory
file_path = "data/taylor_lyrics.csv"

# Read file and print dataframe
data_taylor_lyrics = pd.read_csv(file_path)
data_taylor_lyrics.rename(columns = {'Lyrics':'lyrics', 'Album':'album_name', 'Song Title':'track_name', 'Featured Artists':'featuring', 'Release Date': 'album_release_date'}, inplace=True)
print("\nTaylor Lyrics DataFrame:")
print(data_taylor_lyrics.head())

#Clean data and create new csv
cleaned_lyric_data = data_taylor_lyrics
cleaned_lyric_data.to_csv('data/cleaned_lyric_data.csv', index=False)

# Print column names and data types
print("Column names:", data_taylor_lyrics.columns)
print("Data types:")
print(data_taylor_lyrics.dtypes)


Taylor Lyrics DataFrame:
              album_name album_release_date              track_name featuring  \
0  Taylor Swift (Deluxe)         06/11/2007              Tim McGraw       NaN   
1  Taylor Swift (Deluxe)         06/11/2007         Picture to Burn       NaN   
2  Taylor Swift (Deluxe)         06/11/2007  Teardrops On My Guitar       NaN   
3  Taylor Swift (Deluxe)         06/11/2007   A Place In This World       NaN   
4  Taylor Swift (Deluxe)         06/11/2007             Cold as You       NaN   

                                              lyrics  
0  He said the way my blue eyes shined Put those ...  
1  State the obvious I didn t get my perfect fant...  
2  Drew looks at me I fake a smile so he won t se...  
3  I don t know what I want so don t ask me Cause...  
4  You have a way of coming easily to me And when...  
Column names: Index(['album_name', 'album_release_date', 'track_name', 'featuring',
       'lyrics'],
      dtype='object')
Data types:
album_name           

In [63]:
#Convert data types so 'lyrics' columns are the same in both datasets
cleaned_album_data['lyrics'] = cleaned_album_data['lyrics'].astype(str)
cleaned_lyric_data['lyrics'] = cleaned_lyric_data['lyrics'].astype(str)
data_taylor_album_songs['lyrics'] = data_taylor_album_songs['lyrics'].str.lower()
data_taylor_lyrics['lyrics'] = data_taylor_lyrics['lyrics'].str.lower()

# Merge data and save to new csv
merged_data = pd.merge(cleaned_album_data, cleaned_lyric_data, on='lyrics', how='inner')
merged_data.to_csv('data/merged_taylor_data.csv', index=False)